In [1]:
dataset = load_dataset("code_search_net", "all")
# github for codesearchnet: https://github.com/github/CodeSearchNet#training

NameError: name 'load_dataset' is not defined

In [1]:
from datasets import load_dataset

In [2]:
import pandas as pd
import numpy as np
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

In [3]:
from sentence_transformers import SentenceTransformer, util
import torch

## Directly throw everything in
https://huggingface.co/flax-sentence-embeddings/st-codesearch-distilroberta-base

In [4]:
# model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")
model = SentenceTransformer('all-MiniLM-L6-v2')

In [5]:
sample_train = dataset['train'][:2000]['whole_func_string']

In [9]:
# dataset['train'][10]

In [7]:
# are there labeled dataset and sample queries?
# print(dataset['train'][121]['func_code_string'])

In [8]:
# sample_train[89]

In [12]:
code_emb = model.encode(sample_train, convert_to_tensor=True)

In [15]:
# list of queries: https://github.com/github/CodeSearchNet/blob/master/resources/queries.csv
query = 'convert a date string into yyyymmdd'

In [16]:
query_emb = model.encode(query, convert_to_tensor=True)
hits = util.semantic_search(query_emb, code_emb)[0]
top_hits = hits[:3]

for top_hit in top_hits:
    print("Cossim: {:.2f}".format(top_hit['score']))
    print(sample_train[top_hit['corpus_id']])
    print("\n\n")

Cossim: 0.41
def get_year(self):
        """
        Return the year from the database in the format expected by the URL.
        """
        year = super(BuildableDayArchiveView, self).get_year()
        fmt = self.get_year_format()
        dt = date(int(year), 1, 1)
        return dt.strftime(fmt)



Cossim: 0.40
def get_day(self):
        """
        Return the day from the database in the format expected by the URL.
        """
        year = super(BuildableDayArchiveView, self).get_year()
        month = super(BuildableDayArchiveView, self).get_month()
        day = super(BuildableDayArchiveView, self).get_day()
        fmt = self.get_day_format()
        dt = date(int(year), int(month), int(day))
        return dt.strftime(fmt)



Cossim: 0.40
def get_month(self):
        """
        Return the month from the database in the format expected by the URL.
        """
        year = super(BuildableDayArchiveView, self).get_year()
        month = super(BuildableDayArchiveView, self).g

In [ ]:
# ground truth labels: https://github.com/github/CodeSearchNet/blob/master/resources/annotationStore.csv
# calculate mrr

In [5]:
ruby_data = load_dataset("code_search_net", "ruby")

In [6]:
import pandas as pd

ground_truth_data = pd.read_csv('annotationStore.csv')
ruby_ground_truth = ground_truth_data[ground_truth_data['Language'] == 'Ruby']

In [33]:
ruby_whole_func_string = ruby_data['train']['whole_func_string']
ruby_func_code_url = ruby_data['train']['func_code_url']

In [34]:
ruby_df = pd.DataFrame({'whole_func_string': ruby_whole_func_string, 
                        'func_code_url': ruby_func_code_url})
ruby_df

,whole_func_string,func_code_url
0,def run\n loop do\n break if Actor...,https://github.com/bogdanRada/celluloid_pubsub...
1,def handle_parsed_websocket_message(json_data)...,https://github.com/bogdanRada/celluloid_pubsub...
2,"def unsubscribe(channel, _json_data)\n lo...",https://github.com/bogdanRada/celluloid_pubsub...
3,def delete_server_subscribers(channel)\n ...,https://github.com/bogdanRada/celluloid_pubsub...
4,"def unsubscribe_clients(channel, _json_data)\n...",https://github.com/bogdanRada/celluloid_pubsub...
...,...,...
48786,def on_message(data)\n message = JSON.par...,https://github.com/bogdanRada/celluloid_pubsub...
48787,"def on_close(code, reason)\n connection.t...",https://github.com/bogdanRada/celluloid_pubsub...
48788,"def send_action(action, channel = nil, data = ...",https://github.com/bogdanRada/celluloid_pubsub...
48789,def chat(message)\n final_message = messa...,https://github.com/bogdanRada/celluloid_pubsub...


In [24]:
ruby_ground_truth

,Language,Query,GitHubUrl,Relevance,Notes
3691,Ruby,get name of enumerated value,https://github.com/laskaridis/ruby_enum/blob/e...,0,NaN
3692,Ruby,memoize to disk - persistent memoization,https://github.com/RISCfuture/dropbox/blob/702...,1,NaN
3693,Ruby,socket recv timeout,https://github.com/mojombo/bertrpc/blob/c78721...,3,NaN
3694,Ruby,read properties file,https://github.com/clarete/s3sync/blob/b690957...,2,NaN
3695,Ruby,copying a file to a path,https://github.com/amatsuda/erd/blob/1f4b8bb5d...,1,NaN
...,...,...,...,...,...
4001,Ruby,write csv,https://github.com/khiemns54/sp2db/blob/76c78d...,3,NaN
4002,Ruby,replace in file,https://github.com/gosu/ashton/blob/313673d8b1...,1,NaN
4003,Ruby,how to read the contents of a .gz compressed f...,https://github.com/lostisland/faraday_middlewa...,2,NaN
4004,Ruby,converting uint8 array to image,https://github.com/wvanbergen/chunky_png/blob/...,0,NaN


In [30]:
merged_ruby = ruby_ground_truth.merge(ruby_df, left_on='GitHubUrl', right_on='func_code_url', how='left').reset_index().drop(columns=['Notes', 'index'])

In [32]:
merged_nonnull_ruby = merged_ruby[merged_ruby['func_code_url'].notnull()]
merged_nonnull_ruby

,Language,Query,GitHubUrl,Relevance,whole_func_string,func_code_url
1,Ruby,memoize to disk - persistent memoization,https://github.com/RISCfuture/dropbox/blob/702...,1,def disable_memoization\n @_memoize = fal...,https://github.com/RISCfuture/dropbox/blob/702...
2,Ruby,socket recv timeout,https://github.com/mojombo/bertrpc/blob/c78721...,3,"def connect_to(host, port, timeout = nil)\n ...",https://github.com/mojombo/bertrpc/blob/c78721...
7,Ruby,connect to sql,https://github.com/rsim/ruby-plsql/blob/dc5e74...,2,def connect!(*args)\n params = {}\n ...,https://github.com/rsim/ruby-plsql/blob/dc5e74...
10,Ruby,get current observable value,https://github.com/kontena/kontena/blob/5cb5b4...,2,"def observe(observable, persistent: true)\n ...",https://github.com/kontena/kontena/blob/5cb5b4...
14,Ruby,sorting multiple arrays based on another array...,https://github.com/zendesk/zendesk_api_client_...,0,def join_special_params\n # some params u...,https://github.com/zendesk/zendesk_api_client_...
...,...,...,...,...,...,...
306,Ruby,finding time elapsed using a timer,https://github.com/spox/actiontimer/blob/fcda2...,0,def timer=(t)\n raise ArgumentError.new('...,https://github.com/spox/actiontimer/blob/fcda2...
307,Ruby,matrix multiply,https://github.com/Nephos/epimath100/blob/a415...,2,def *(matrix)\n #produit matriciel\n #co...,https://github.com/Nephos/epimath100/blob/a415...
309,Ruby,converting uint8 array to image,https://github.com/jellymann/mittsu/blob/955bd...,1,"def render(scene, camera)\n return if @sp...",https://github.com/jellymann/mittsu/blob/955bd...
311,Ruby,replace in file,https://github.com/gosu/ashton/blob/313673d8b1...,1,def replace_include(source)\n source.gsub...,https://github.com/gosu/ashton/blob/313673d8b1...


In [27]:
ruby_train = ruby_data['train']['whole_func_string']

In [35]:
code_emb = model.encode(ruby_train, convert_to_tensor=True)

KeyboardInterrupt: 

In [36]:
query = 'replace in file'
query_emb = model.encode(query, convert_to_tensor=True)

hits = util.semantic_search(query_emb, code_emb)[0]
top_hits = hits[:3]

for top_hit in top_hits:
    print("Cossim: {:.2f}".format(top_hit['score']))
    print(ruby_train[top_hit['corpus_id']])
    print("\n\n")

Cossim: 0.61
def modify_content(source, content, replace_value)
      file = File.read(source)
      replace = file.gsub(/#{content}/, replace_value)
      File.open(source, "w"){|f|
        f.puts replace       
      }
    end



Cossim: 0.56
def replace_file(original_file_object=nil,
                         replacement_file_path='',
                         retain_original_filename_in_oa=false,
                         generate_objects=false)

            __replace_file(original_file_object,
                         replacement_file_path,
                         retain_original_filename_in_oa,
                         generate_objects)
        end



Cossim: 0.52
def replace_in_file(relative_path, *args, **options, &block)
      check_path(relative_path)
      contents = ::File.read(relative_path)
      replacement = (block ? block[] : args[1..-1].join).gsub('\0', '')
      match = Regexp.escape(replacement)
      status = nil

      log_status(:replace, relative_path, options.fet

In [50]:
ruby_ground_truth

,Language,Query,GitHubUrl,Relevance,Notes
3691,Ruby,get name of enumerated value,https://github.com/laskaridis/ruby_enum/blob/e...,0,NaN
3692,Ruby,memoize to disk - persistent memoization,https://github.com/RISCfuture/dropbox/blob/702...,1,NaN
3693,Ruby,socket recv timeout,https://github.com/mojombo/bertrpc/blob/c78721...,3,NaN
3694,Ruby,read properties file,https://github.com/clarete/s3sync/blob/b690957...,2,NaN
3695,Ruby,copying a file to a path,https://github.com/amatsuda/erd/blob/1f4b8bb5d...,1,NaN
...,...,...,...,...,...
4001,Ruby,write csv,https://github.com/khiemns54/sp2db/blob/76c78d...,3,NaN
4002,Ruby,replace in file,https://github.com/gosu/ashton/blob/313673d8b1...,1,NaN
4003,Ruby,how to read the contents of a .gz compressed f...,https://github.com/lostisland/faraday_middlewa...,2,NaN
4004,Ruby,converting uint8 array to image,https://github.com/wvanbergen/chunky_png/blob/...,0,NaN


In [17]:
y_true = merged_nonnull_ruby['Relevance']
y_true

1      1
2      3
7      2
10     2
17     2
      ..
306    0
307    2
309    1
311    1
313    0
Name: Relevance, Length: 109, dtype: int64